In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to silver
# dbutils.fs.mount(
# 	source = "wasbs://silver@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/silver",
# 	extra_configs = configs
# )

In [0]:
df_language = spark.read.csv("/mnt/datalake/model/language.csv", header=True, mode="overwrite")
df_language.show()

+---+---------+----------------+
| id|lang_code|       lang_name|
+---+---------+----------------+
|  1|       en|         English|
|  2|       vi|      Vietnamese|
|  3|       nb|Norwegian Bokmal|
|  4|       sl|       Slovenian|
|  5|       ro|        Romanian|
|  6|       ka|        Georgian|
|  7|       ur|            Urdu|
|  8|       lv|         Latvian|
|  9|       pl|          Polish|
| 10|       cn|         Chinese|
| 11|       sk|          Slovak|
| 12|       ay|          Aymara|
| 13|       kk|          Kazakh|
| 14|       pt|      Portuguese|
| 15|       tl|         Tagalog|
| 16|       sh|  Serbo Croatian|
| 17|       ko|          Korean|
| 18|       ms|           Malay|
| 19|       uk|       Ukrainian|
| 20|       jv|        Javanese|
+---+---------+----------------+
only showing top 20 rows



In [0]:
from delta.tables import *
from pyspark.sql.functions import col, substring

# #check if the silver location contain the delta table
# if (DeltaTable.isDeltaTable(spark, movie_silverLocation)):
#     DeltaTable.forPath(spark, movie_silverLocation).alias("target").merge(
#         source = cleaned_movie.alias("src"),
#         condition = "target.id = src.id"
#     ).WhenNoMatchedInsertAll().execute()
# else:

# Define the silver location

# Define the silver location
language_silverLocation = "/mnt/datalake/silver/cleaned_language"

# Table name
language_silverTable = "silver.language"

df_language.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(language_silverLocation)


In [0]:
# create the schema and table, if required
# spark.databricks.delta.formatCheck.enabled false

spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {language_silverTable} USING delta LOCATION '{language_silverLocation}'")


DataFrame[]

In [0]:
# Maintenance for Delta table
language_DataDelta = DeltaTable.forName(spark, language_silverTable)

if language_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	language_DataDelta.optimize()
	language_DataDelta.vacuum()


In [0]:
%sql
DESCRIBE HISTORY silver.company;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2023-12-17T09:00:54Z,1311347395691033,totnvt02@gmail.com,VACUUM END,Map(status -> COMPLETED),null,List(3918195481499330),1122-230829-ksqo7xkx,1,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/13.3.x-scala2.12
1,2023-12-17T09:00:47Z,1311347395691033,totnvt02@gmail.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(3918195481499330),1122-230829-ksqo7xkx,0,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/13.3.x-scala2.12
0,2023-12-17T08:53:52Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(3918195481499330),1122-230829-ksqo7xkx,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 22027, numOutputBytes -> 393476)",null,Databricks-Runtime/13.3.x-scala2.12


In [0]:
%sql
select * from silver.language

id,lang_code,lang_name
1,en,English
2,vi,Vietnamese
3,nb,Norwegian Bokmal
4,sl,Slovenian
5,ro,Romanian
6,ka,Georgian
7,ur,Urdu
8,lv,Latvian
9,pl,Polish
10,cn,Chinese
